In [9]:
import pandas as pd

books = pd.read_csv('../data/Books.csv', delimiter=';', low_memory=False)
ratings = pd.read_csv('../data/Ratings.csv', delimiter=';')
users = pd.read_csv('../data/Users.csv', delimiter=';')


C:\Users\Никита\AppData\Local\Temp\ipykernel_13268\3653110595.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('../data/Users.csv', delimiter=';')


In [6]:
df_prep_step_1 = pd.merge(books, ratings.query("`Rating` > 0"), on='ISBN', how='inner')
df_prep_step_2 = pd.merge(df_prep_step_1, users, on='User-ID', how='inner')
df_prep = df_prep_step_2.drop(['Publisher'], axis=1)
df_isbn = df_prep.drop_duplicates()
df_isbn['Original_NaN'] = df_isbn['Age'].isna()
df_isbn['Age'] = pd.to_numeric(df_isbn['Age'], errors='coerce')
df_isbn = df_isbn[~(users['Age'].isna() & ~df_isbn['Original_NaN'])]
df_isbn.drop(columns=['Original_NaN'], inplace=True)
df_isbn['User-ID'] = pd.to_numeric(df_isbn['User-ID'], errors='coerce')
df_isbn = df_isbn[~(df_isbn['User-ID'].isna())]
age_outliers = df_isbn.query("Age > 100 or Age < 6")
user_outliers = age_outliers["User-ID"].to_list()
df_isbn = df_isbn[~df_isbn["User-ID"].isin(user_outliers)]

C:\Users\Никита\AppData\Local\Temp\ipykernel_13268\3869774476.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isbn['Original_NaN'] = df_isbn['Age'].isna()
C:\Users\Никита\AppData\Local\Temp\ipykernel_13268\3869774476.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isbn['Age'] = pd.to_numeric(df_isbn['Age'], errors='coerce')
C:\Users\Никита\AppData\Local\Temp\ipykernel_13268\3869774476.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_isbn = df_isbn[~(users[

In [ ]:
from sklearn.model_selection import train_test_split

y = df_isbn['Rating']
X = df_isbn.drop('Rating', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [8]:
from src.recommenders.als_recommender import ALSRecommender

ratings = df_isbn[['User-ID', 'ISBN', 'Rating']]
books = df_isbn[['ISBN', 'Title', 'Author', 'Year']]
users = df_isbn[['User-ID', 'Age']]

als = ALSRecommender()
als.fit(ratings, books, users)

metrics = als.eval(ratings)
print(metrics)

        User-ID        ISBN  Rating
0             8  0002005018     5.0
1         11676  0002005018     8.0
3        116866  0002005018     9.0
4        123629  0002005018     9.0
5        200273  0002005018     8.0
...         ...         ...     ...
363529   259901  0679861548     7.0
363530   259901  0816705739    10.0
363531   259901  0822502763    10.0
363532   259901  5550534274    10.0
363533   260239  3404128907     8.0

[285506 rows x 3 columns]


D:\Новая папка1\rs-ucu-books-recommender\myenv\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns
D:\Новая папка1\rs-ucu-books-recommender\myenv\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns
D:\Новая папка1\rs-ucu-books-recommender\myenv\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [9]:
ratings

,User-ID,ISBN,Rating
0,8,0002005018,5
1,11676,0002005018,8
3,116866,0002005018,9
4,123629,0002005018,9
5,200273,0002005018,8
...,...,...,...
363529,259901,0679861548,7
363530,259901,0816705739,10
363531,259901,0822502763,10
363532,259901,5550534274,10
